beomi/llama-2-ko-7b

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer
from peft import PeftModel, PeftConfig


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/brianjang7/home1/.venv/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda112.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 112
CUDA SETUP: Loading binary /home/brianjang7/home1/.venv/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda112.so...


In [2]:
!nvidia-smi

Thu Jul 27 10:38:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 50%   41C    P8    29W / 350W |     20MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:4A:00.0 Off |                  N/A |
|  0%   

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "beomi/llama-2-ko-7b"  # safetensors 컨버팅된 레포
bnb_config = BitsAndBytesConfig(    # 모델의 성능을 유지하면서 메모리 사용을 최적화하고, 하드웨어 환경에 맞게 데이터를 처리하는 데 도움
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [4]:
from peft import prepare_model_for_kbit_training
# 모델의 훈련 과정에서 필요한 메모리 양을 줄일 수 있습니다.
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [5]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [6]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [7]:
gen('건강하게 살기 위한 세 가지 방법은?')

/home/brianjang7/home1/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


<s> ### 질문: 건강하게 살기 위한 세 가지 방법은?

### 답변: 누구나 건강한 삶을 누리고 싶어 한다. 하지만 건강하게 살려면 어떻게 해야 할까? 전문가들은 건강 유지와 증진을 위해 적당한 음식을 먹거나 운동을 하는 것이 중요하지만, 이보다 더욱 중요한 것으로 다음과 같이 세 가지를 들고 있다. 바로 규칙적인 생활습관, 규칙적으로 운동하는 습관, 체내에 쌓인 독성 배출을 위한 노력이 그것이다. 먼저 체내에 독성물질을 제거하기 위해 규칙적으로 해독 주스나 녹즙을 마시거나, 채소 주스/생즙을 하루 4회 이상, 오메가3와 유산균을 규칙적으로 섭취를 하라. 또한 유기농 자연곡물로 식단을 꾸려라. 규칙적인 생활습관은 숙면에 도움이 되는 생체리듬을 가질 수 있게 해준다. 이밖에 규칙적인 운동습관을 가지면, 심혈관, 우울증, 두뇌 건강에 좋다는 최근의 연구 결과에 따르면 수면의 질 또한 크게 향상할 수 있다고 한다. 한 연구에 따르면, 규칙적인 운동을 한 성인 백 명이 3주 동안 실험을 한 결과, 87%가 수면의 질이 향상되고, 11%는 불안증이나 부정적인 생각이 사라지고, 12%는 우울
